In [2]:
from pathlib import Path
import pandas as pd

In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.vectorstores import Chroma

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import os

os.environ['OPENAI_API_KEY'] = 'sk-proj-3pSCJqVVn8lRccIMTfXFT3BlbkFJvB0pdZtI3qOHc4EmWKKV'


def initialize_model(model_name):
    if model_name == 'gpt-4o-mini':
        return ChatOpenAI(model=model_name)
    elif model_name == 'claude':
        return ChatAnthropic(model=model_name)
    else:
        raise ValueError("Unsupported model name. Choose either 'gpt-4o-mini' or 'claude'.")


In [2]:
def complete_todo_list(input_text, model_name='gpt-4o-mini'):
    # モデルの初期化
    model = initialize_model(model_name)
    # ベクトルデータベースの読み込み
    database = Chroma(
            persist_directory="./.data", 
            embedding_function=OpenAIEmbeddings(model="text-embedding-ada-002")
        )
    # 文字列を1行ずつ分割
    lines = input_text.split('\n')
    # 結果を格納する変数
    concatenated_text = ""
    for line in lines:
        docs = database.similarity_search(line)
        print(docs)
        # プロンプトテンプレートの定義
        prompt_template = "タスクに業務マニュアルに沿った情報を補完せよ。\
                        補完時の注意事項 \
                        1. 「タスク名(氏名)」の下の行に、「※補足：」から始まる形で業務マニュアルに沿った補足を追記すること。\
                        :\n\n{line} \
                        \n\n関連業務マニュアル:{context}"
        # メッセージの作成
        messages = [
            SystemMessage(content="あなたはプロジェクトリーダーです。業務マニュアルに沿った補足情報を追加して、部下にわかりやすくタスクを振ることができます。"),
            HumanMessage(content=prompt_template.format(line=line, context=docs[0].content))
        ]
        # 生成AIモデルから返事を取得
        completed_todo = model.invoke(messages)
        concatenated_text += completed_todo.choices[0].text + "\n"
    return concatenated_text

In [3]:
input_text = "太郎さんに議事録を作成してもらう\n次回の会議資料を作成する\n"
completed_text = complete_todo_list(input_text)

c:\Users\sakih\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\sakih\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


RuntimeError: Chroma is running in http-only client mode, and can only be run with 'chromadb.api.fastapi.FastAPI' or 'chromadb.api.async_fastapi.AsyncFastAPI' as the chroma_api_impl.             see https://docs.trychroma.com/guides#using-the-python-http-only-client for more information.